In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(color_codes=True)
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, roc_curve, plot_roc_curve
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import RidgeClassifier
import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.rcParams['axes.labelsize'] = 15.
plt.rcParams['xtick.labelsize'] = 15.
plt.rcParams['ytick.labelsize'] = 15.
plt.rcParams['legend.fontsize'] = 15.
plt.rcParams['figure.figsize'] = [15.,8.]

In [ ]:
df = pd.read_csv('../input/heart-disease-uci/heart.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
sns.countplot(data=df, x="target", palette="magma");

In [ ]:
for i in df.columns[:13]:
    sns.histplot(data=df, x=i, palette='magma')
    plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df.age[df.target==0],
           df.cp[df.target==0],
           c='g')
plt.scatter(df.age[df.target==1],
           df.cp[df.target==1],
           c="b")
plt.title("Age vc CP for Heart Disease")
plt.legend(["No Disease","aving Hneart Disease"])
plt.xlabel("Age")
plt.ylabel("Chest Pain");

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df.age[df.target==0],
           df.thalach[df.target==0],
           c="g")
plt.scatter(df.age[df.target==1],
           df.thalach[df.target==1],
           c="b")
plt.xlabel("Age")
plt.ylabel("Thalach")
plt.legend(["No Disease", "Disease"])
plt.title("Age vs Thalach for Heart Disease");

In [ ]:
for i in df.columns[:14]:
    sns.boxplot(data=df, x=i)
    plt.show()

In [ ]:
outliers_col = ["trestbps", "chol","fbs","thalach", "oldpeak", "ca","thal"]

In [ ]:
for i in outliers_col[:]:
    IQR = df[i].quantile(0.75)-df[i].quantile(0.25)
    lower_bridge=df[i].quantile(0.25)-(IQR*1.5)
    upper_bridge=df[i].quantile(0.75)+(IQR*1.5)
    print(f' {i} : Lower Bridge: {lower_bridge}, Upper Bridge: {upper_bridge}')

In [ ]:
df.loc[df['trestbps']>=170, 'trestbps']=170
df.loc[df['trestbps']<=90, 'trestbps']=90

df.loc[df['chol']>=369.75, 'chol']=369.75
df.loc[df['chol']<=115.75, 'chol']=115.75

df.loc[df['fbs']>=0, 'fbs']=0
df.loc[df['fbs']<=0, 'fbs']=0

df.loc[df['thalach']>=214.75, 'thalach']=214.75
df.loc[df['thalach']<=84.75, 'thalach']=84.75

df.loc[df['oldpeak']>=4.0, 'oldpeak']=4.0
df.loc[df['oldpeak']<=-2.40, 'oldpeak']=-2.40

df.loc[df['ca']>=2.5, 'ca']=2.5
df.loc[df['ca']<=-1.5, 'ca']=-1.5

df.loc[df['thal']>=4.5, 'thal']=4.5
df.loc[df['thal']<=0.5, 'thal']=0.5

In [ ]:
for i in df.columns[:14]:
    sns.boxplot(data=df, x=i)
    plt.show()

### Outliers are removed

In [ ]:
sns.pairplot(data=df)

In [ ]:
corr = df.corr()
corr

In [ ]:
sns.heatmap(corr,
           cmap="Accent",
           annot=True);

In [ ]:
df.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
standard = StandardScaler()
features_to_scale= [feature for feature in df.columns if feature not in ['target']]
df[features_to_scale] = standard.fit_transform(df[features_to_scale])

In [ ]:
df.head()

In [ ]:
# train test split
x = df.drop(['target'], axis=1)
y = df['target']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42)

In [ ]:
len(x_train), len(y_train)

In [ ]:
df.shape

## Base Model

### RandomForestClassifier

In [ ]:
rf =  RandomForestClassifier().fit(x_train,y_train)
rf.score(x_train,y_train)

In [ ]:
rf.score(x_test,y_test)

In [ ]:
y_preds = rf.predict(x_test)
accuracy_score(y_test,y_preds)

In [ ]:
sns.heatmap(confusion_matrix(y_test,y_preds), annot=True);

In [ ]:
acc_cv = cross_val_score(rf, x,y, scoring="accuracy", cv=5, n_jobs=1)

In [ ]:
acc_cv.mean()

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,y_preds))

In [ ]:
plot_roc_curve(rf,x_test,y_test)

### Logistic Regression

In [ ]:
lr = LogisticRegression()
lr.fit(x_train,y_train)
lr.score(x_train,y_train)

In [ ]:
y_preds1= lr.predict(x_test)
print(f'Accuracy Score: {accuracy_score(y_test,y_preds1)}')
sns.heatmap(confusion_matrix(y_test,y_preds1), annot=True)
print(classification_report(y_test,y_preds1))
print(plot_roc_curve(lr, x_test,y_test))

### AdaBoostClassifier

In [ ]:
ad = AdaBoostClassifier().fit(x_train,y_train)
y_preds2 = ad.predict(x_test)
print(f'Accuracy Score: {accuracy_score(y_test,y_preds2)}')
sns.heatmap(confusion_matrix(y_test,y_preds2), annot=True)
print(classification_report(y_test,y_preds2))
print(plot_roc_curve(lr, x_test,y_test))

### KNearestNeighbor Classifier

In [ ]:
knn = KNeighborsClassifier().fit(x_train,y_train)
y_preds3 = knn.predict(x_test)
print(f'Accuracy Score: {accuracy_score(y_test,y_preds3)}')
sns.heatmap(confusion_matrix(y_test,y_preds3), annot=True)
print(classification_report(y_test,y_preds3))
print(plot_roc_curve(lr, x_test,y_test))

### RidgeClassifier

In [ ]:
knn = RidgeClassifier().fit(x_train,y_train)
y_preds4 = knn.predict(x_test)
print(f'Accuracy Score: {accuracy_score(y_test,y_preds4)}')
sns.heatmap(confusion_matrix(y_test,y_preds4), annot=True)
print(classification_report(y_test,y_preds4))
print(plot_roc_curve(lr, x_test,y_test))

## Hyperparameter Tuning 

In [ ]:
param_grid = [    
    {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100, 1000,2500, 5000]
    }
]
lr_random = RandomizedSearchCV(lr, 
                              param_distributions=param_grid,
                              cv=5,
                              n_iter=100,
                              verbose=2,
                              n_jobs=-1,
                              random_state=42)
lr_random.fit(x_train,y_train)

In [ ]:
lr_random.best_params_

In [ ]:
lr1_random = lr_random.best_params_

In [ ]:
lr1_random

In [ ]:
log = LogisticRegression(solver ='newton-cg', penalty='l2',max_iter= 100, C= 0.08858667904100823)

In [ ]:
log.fit(x_train,y_train)

In [ ]:
log.score(x_train,y_train)

In [ ]:
y_pred = log.predict(x_test)
print(f'Accuracy Score: {accuracy_score(y_test,y_pred)}')
sns.heatmap(confusion_matrix(y_test,y_preds4), annot=True)
print(classification_report(y_test,y_preds4))
print(plot_roc_curve(lr, x_test,y_test))